In [ ]:
import pandas as pd
import scipy.stats as st
import seaborn as sns
import plotly.express as px
import numpy as np
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportions_ztest
import datetime as dt

# EDA

In [ ]:
data = pd.read_csv("Контест_Аха_Самокат_тех_данные_по_мошенникам.csv", sep = ";")
# reorder
data = data[['merchant_id', 'registration_date', 'activation_date', 'type', 'ind_frod']]
# change the format
data['registration_date'] = pd.to_datetime(data['registration_date'], format='%d.%m.%Y')
data['activation_date'] = pd.to_datetime(data['activation_date'], format='%d.%m.%Y')
# substitute  teh nan values in teh activation date column
data[data["activation_date"].isnull()] = 0
data

,merchant_id,registration_date,activation_date,type,ind_frod
0,1,2023-12-16 00:00:00,2023-12-24 00:00:00,IE,0.0
1,2,2023-09-05 00:00:00,2023-09-08 00:00:00,IE,0.0
2,3,2023-04-16 00:00:00,2023-04-22 00:00:00,IE,0.0
3,4,2023-12-23 00:00:00,2023-12-24 00:00:00,IE,0.0
4,0,0,0,0,0.0
...,...,...,...,...,...
34995,34996,2023-10-20 00:00:00,2022-10-31 00:00:00,LLC,0.0
34996,34997,2023-04-20 00:00:00,2023-04-22 00:00:00,LLC,0.0
34997,34998,2023-08-24 00:00:00,2023-09-04 00:00:00,LLC,0.0
34998,34999,2023-06-08 00:00:00,2023-06-18 00:00:00,LLC,0.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   merchant_id        35000 non-null  int64  
 1   registration_date  35000 non-null  object 
 2   activation_date    35000 non-null  object 
 3   type               35000 non-null  object 
 4   ind_frod           34601 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.3+ MB


In [ ]:
data.ind_frod.value_counts()

ind_frod
0.0    32990
1.0     1611
Name: count, dtype: int64

In [ ]:
data.type.value_counts()

type
0      14700
IE     14612
LLC     5688
Name: count, dtype: int64

In [ ]:
# let's look at the nan values
data.isnull().mean()

merchant_id          0.0000
registration_date    0.0000
activation_date      0.0000
type                 0.0000
ind_frod             0.0114
dtype: float64

In [ ]:
data[data["ind_frod"].isnull()]

,merchant_id,registration_date,activation_date,type,ind_frod
138,139,2023-03-12 00:00:00,2023-03-20 00:00:00,IE,NaN
333,334,2023-01-31 00:00:00,2023-02-10 00:00:00,IE,NaN
341,342,2023-03-07 00:00:00,2023-03-12 00:00:00,IE,NaN
489,490,2023-01-13 00:00:00,2023-01-23 00:00:00,IE,NaN
541,542,2023-02-28 00:00:00,2023-03-07 00:00:00,IE,NaN
...,...,...,...,...,...
34694,34695,2023-11-16 00:00:00,2023-11-20 00:00:00,LLC,NaN
34768,34769,2023-05-17 00:00:00,2023-05-26 00:00:00,LLC,NaN
34779,34780,2023-05-10 00:00:00,2023-05-16 00:00:00,LLC,NaN
34982,34983,2023-12-30 00:00:00,2024-01-09 00:00:00,LLC,NaN


In [ ]:
# null values in the activation date
count = (data["activation_date"] == 0).sum()
print(count)

14700


In [ ]:
# Filter rows where activation_date is 0 and check if ind_frod is NaN
is_nan = data[data['activation_date'] == 0]['ind_frod'].isna().all()
if is_nan:
    print("When activation_date is 0, the corresponding values in ind_frod are NaN.")
else:
    print("When activation_date is 0, the corresponding values in ind_frod are not all NaN.")

When activation_date is 0, the corresponding values in ind_frod are not all NaN.


In [ ]:
data.head(2)

,merchant_id,registration_date,activation_date,type,ind_frod
0,1,2023-12-16 00:00:00,2023-12-24 00:00:00,IE,0.0
1,2,2023-09-05 00:00:00,2023-09-08 00:00:00,IE,0.0


In [ ]:
data_grouped = data.groupby('type')
print(data_grouped)

In [ ]:
# the percentage of missing values in each group
count_IE = 0
count_LLC = 0
total_IE = 0
total_LLC = 0

for name, group in data.groupby('type'):
    total = len(group)  # Total count of rows in the group
    missing_count = group['ind_frod'].isna().sum()  # Count of missing values in the group

    if name == 'IE':
        total_IE += total
        count_IE += missing_count
    elif name == 'LLC':
        total_LLC += total
        count_LLC += missing_count

percentage_missing_IE = (count_IE / total_IE) * 100
percentage_missing_LLC = (count_LLC / total_LLC) * 100

print(f"Percentage of missing values in IE group: {percentage_missing_IE:.2f}%")
print(f"Percentage of missing values in LLC group: {percentage_missing_LLC:.2f}%")

Percentage of missing values in IE group: 1.96%
Percentage of missing values in LLC group: 1.99%


In [ ]:
count_IE

286

In [ ]:
# Create histograms for each group using Plotly Express
figs = []
for name, group in data.groupby('type'):
    fig = px.histogram(group, x='ind_frod', title=f'Ind_frod Counts for Type {name}',
                       category_orders={'ind_frod': [0, 1]})
    figs.append(fig)

# Show the histograms
for fig in figs:
    fig.show()

In [ ]:
data

,merchant_id,registration_date,activation_date,type,ind_frod
0,1,2023-12-16 00:00:00,2023-12-24 00:00:00,IE,0.0
1,2,2023-09-05 00:00:00,2023-09-08 00:00:00,IE,0.0
2,3,2023-04-16 00:00:00,2023-04-22 00:00:00,IE,0.0
3,4,2023-12-23 00:00:00,2023-12-24 00:00:00,IE,0.0
4,0,0,0,0,0.0
...,...,...,...,...,...
34995,34996,2023-10-20 00:00:00,2022-10-31 00:00:00,LLC,0.0
34996,34997,2023-04-20 00:00:00,2023-04-22 00:00:00,LLC,0.0
34997,34998,2023-08-24 00:00:00,2023-09-04 00:00:00,LLC,0.0
34998,34999,2023-06-08 00:00:00,2023-06-18 00:00:00,LLC,0.0


In [ ]:
data.ind_frod.value_counts()

ind_frod
0.0    32990
1.0     1611
Name: count, dtype: int64

# Расчет Размера выборки

Для расчета размера выборки на основе данных о регистрациях и активациях продавцов, нам потребуется использовать статистические методы. Один из подходов для определения размера выборки для A/B-тестов заключается в использовании известных формул для сравнения двух долей. Здесь мы будем учитывать метрику индекса мошенничества (ind_frod).

**Параметры A/B-теста и Расчет Размеров Выборки**

**Введение**

Для проведения A/B-теста важно понимать, как выбор параметров \(\alpha\), \(\beta\), MDE и мощности теста влияет на размер выборки и результаты теста.

**Альфа**

Альфа — это уровень значимости теста. Это вероятность отвергнуть нулевую гипотезу, когда она истинна (ошибка первого рода). Обычно устанавливается на уровне 0.05, что означает 5\% вероятность ложного обнаружения эффекта.

**Бета**

Бета — это вероятность не отвергнуть нулевую гипотезу, когда альтернативная гипотеза истинна (ошибка второго рода). Бета связана с мощностью теста как **мощность** = 1 - Бета.

**Минимальный Обнаруживаемый Эффект (MDE)**

Минимальный Обнаруживаемый Эффект (MDE) — это наименьший размер эффекта, который мы хотим обнаружить в тесте. MDE определяется как разница в конверсии между контрольной и экспериментальной группами, которую мы считаем значимой.

**Мощность теста**

Мощность теста — это вероятность обнаружить эффект, если он действительно существует. Мощность обычно устанавливается на уровне 0.8 или 80\%, что означает 20\% вероятность ошибки второго рода (не обнаружить существующий эффект).

**Пример расчета размера выборки**

В данном примере мы будем использовать следующие параметры:
alpha = 0.05
мощность = 0.8 (соответствует \(\beta = 0.2\))
MDE = 0.1 (ожидаемый эффект, 10\%)


In [ ]:
# Фильтруем данные, убирая строки с нулевыми значениями
data = data[data['merchant_id'] != 0]

# Определим базовые параметры
baseline_conversion_rate = data['ind_frod'].mean()  # Средняя доля мошенников в контрольной группе
effect_size = 0.1  # Ожидаемый эффект (разница между контрольной и экспериментальной группами)
alpha = 0.05  # Уровень значимости
power = 0.8  # Желаемая мощность теста

# Рассчитаем размер выборки
power_analysis = NormalIndPower()
sample_size = power_analysis.solve_power(effect_size=effect_size,
                                         alpha=alpha,
                                         power=power,
                                         ratio=1,  # Соотношение размеров контрольной и экспериментальной групп
                                         alternative='larger')

# Учитывая обе группы
total_sample_size = sample_size * 2

print(f"Необходимый размер выборки для каждой группы: {int(np.ceil(sample_size))}")
print(f"Общий размер выборки: {int(np.ceil(total_sample_size))}")


Необходимый размер выборки для каждой группы: 1237
Общий размер выборки: 2474


Определение Статистического критерия для теста

# Проведение A/A-тестов и Определение Критерия для A/B-теста

При проведении A/A-тестов основной целью является проверка корректности системы A/B-тестирования и того, что методология тестирования не приводит к ложноположительным результатам. В A/A-тестах мы делим аудиторию на две равные группы, которые получают идентичное лечение, и проверяем, что никакой значимой разницы между ними нет.

## Определение критерия для A/B-теста

Основные критерии, которые используются для сравнения долей или средних значений в A/B-тестах, включают:

- **Z-тест для пропорций**: Используется для сравнения долей, например, доли жалоб в контрольной и экспериментальной группах.
- **T-тест для независимых выборок**: Используется для сравнения средних значений между двумя независимыми группами.
- **Chi-квадрат тест**: Используется для проверки зависимости между двумя категориальными переменными.

В контексте проверки доли жалоб (Customer Complaint Rate), наиболее подходящим будет использование Z-теста для пропорций.

## Проведение A/A-теста

Проведение A/A-теста включает следующие шаги:

1. **Сбор данных**: Разделите вашу аудиторию на две группы и соберите данные по каждой группе.
2. **Статистический тест**: Проведите Z-тест для пропорций, чтобы убедиться, что между двумя группами нет значимой разницы.
3. **Проверка значимости**: Убедитесь, что p-value больше уровня значимости \(\alpha\) (обычно 0.05), что подтверждает отсутствие значимой разницы.

## CUPED

Для повышения точности оценки эффекта в A/B-тестах можно использовать метод линеаризации метрик. Один из таких методов — **CUPED** (Controlled Using Pre-Experiment Data). CUPED использует предварительные данные для уменьшения дисперсии метрики, что увеличивает статистическую мощность теста.

### Основная идея CUPED

CUPED включает создание скорректированной метрики, которая учитывает предтестовые значения. Это делается путем регрессии основной метрики на предтестовую метрику и корректировки с использованием остатков этой регрессии. Это позволяет уменьшить дисперсию и повысить точность оценки эффекта.

## Использование машинного обучения для снижения дисперсии

Машинное обучение может быть использовано для предсказания значений метрики на основе различных предикторов. Это позволяет создавать скорректированные метрики, которые имеют меньшую дисперсию и обеспечивают более точные оценки эффекта.




In [ ]:
# Генерация данных для A/A-теста
np.random.seed(42)
n = 10000  # Размер выборки каждой группы
p = 0.05  # Ожидаемая доля жалоб

# Создаем две группы с одинаковой долей жалоб
group_A = np.random.binomial(1, p, n)
group_B = np.random.binomial(1, p, n)

# Подсчет числа жалоб в каждой группе
complaints_A = np.sum(group_A)
complaints_B = np.sum(group_B)

# Размеры выборок
n_A = len(group_A)
n_B = len(group_B)

# Проведение Z-теста для пропорций
count = np.array([complaints_A, complaints_B])
nobs = np.array([n_A, n_B])
z_stat, p_value = proportions_ztest(count, nobs)

print(f"Z-статистика: {z_stat}")
print(f"P-значение: {p_value}")

# Проверка значимости
alpha = 0.05
if p_value > alpha:
    print("Нет значимой разницы между группами (A/A-тест пройден).")
else:
    print("Значимая разница обнаружена (A/A-тест не пройден).")


Z-статистика: -1.655442882082843
P-значение: 0.09783469357634281
Нет значимой разницы между группами (A/A-тест пройден).
